# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

# Цель исследования

Разработать модель машинного обучения для определения рыночной стоимости автомобиля, основываясь на его характеристиках и комплектации.

***Ожидаемый результат:***

- Точная оценка стоимости автомобиля для клиентов сервиса.
- Снижение разницы между предсказанной и реальной ценой.
- Быстрая работа модели, обеспечивающая мгновенный отклик в приложении.

# Суть исследования

Создать и протестировать различные модели машинного обучения для прогнозирования цены автомобилей с пробегом.

#  Задачи исследования

***Этап 1. Подготовка данных***

***Этап 2. Обучение моделей***

***Этап 3. Анализ результатов***


# Исходные данные

***Признаки***

- `DateCrawled` — дата скачивания анкеты из базы
- `VehicleType` — тип автомобильного кузова
- `RegistrationYear` — год регистрации автомобиля
- `Gearbox` — тип коробки передач
- `Power` — мощность (л. с.)
- `Model` — модель автомобиля
- `Kilometer` — пробег (км)
- `RegistrationMonth` — месяц регистрации автомобиля
- `FuelType` — тип топлива
- `Brand` — марка автомобиля
- `Repaired` — была машина в ремонте или нет
- `DateCreated` — дата создания анкеты
- `NumberOfPictures` — количество фотографий автомобиля
- `PostalCode` — почтовый индекс владельца анкеты (пользователя)
- `LastSeen` — дата последней активности пользователя


***Целевой признак***
- `Price` — цена (евро)

In [1]:
!pip install numpy==1.21.6 pandas==1.3.5 scipy==1.7.3 scikit-learn==1.1.3 category_encoders==2.6.0

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

import lightgbm as lgb
from catboost import CatBoostRegressor

import time

## Подготовка данных

In [3]:
# Загрузка данных с проверкой
try:
    df = pd.read_csv('/datasets/autos.csv')
except:
    df = pd.read_csv('autos.csv')

In [4]:
# Функция для вывода общей информации
def info_data(df):
    display(df.info())
    display(df.describe())

In [5]:
info_data(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

None

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [6]:
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [7]:
# Функция для вывода кол-ва дубликатов и пропусков
def search_losses(df):
    print(f"Количество дубликатов: {df.duplicated().sum()}\n")
    print(f"Количество пропусков:\n{df.isna().sum()}\n")

In [8]:
search_losses(df)

Количество дубликатов: 4

Количество пропусков:
DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
Repaired             71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64



***1. Пропущенные значения:*** 

- Встречаются в колонках `VehicleType`, `Model`, `FuelType`, `Repaired`.

***2. Некорректные значения:***

- `Power` содержит нулевые значения, что маловероятно для автомобилей.
- `RegistrationYear` может включать аномальные значения (например, выше текущего года).
- `RegistrationMonth` содержит 0, что тоже странно.

***3. Пропуски:***

- `VehicleType` (37490)
- `Gearbox` (19833)
- `Model` (19705)
- `FuelType` (32895)
- `Repaired` (71154)

***Для категориальных данных можно заполнить пропуски модой или добавить категорию "unknown".***

***4. Аномальные значения:***

- `Power`: от 0 до 20 000 л.с., что явно нереалистично.
- `RegistrationYear`: есть значения от 1000 года.
- `RegistrationMonth`: есть 0 (регистрация должна быть в одном из 12 месяцев).

***5. Неинформативные признаки:***

- `DateCrawled`, `DateCreated`, `LastSeen` (уберём).

- `NumberOfPictures` (везде 0, не даёт информации).

- `PostalCode` (не влияет на цену, тоже уберём).

In [9]:
# Удаление неинформативных признаков
df.drop(columns=['DateCrawled', 'DateCreated', 'LastSeen', 'NumberOfPictures', 'PostalCode'], inplace=True)

In [10]:
# Обрабатка пропусков
categorical_columns = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Repaired']
df[categorical_columns] = df[categorical_columns].fillna('unknown')

In [11]:
# Фильтруем аномальные значения в RegistrationYear
df = df[(df['RegistrationYear'] >= 1900) & (df['RegistrationYear'] <= 2016)]

In [12]:
# Фильтруем нулевые и слишком большие значения в Power
df = df[(df['Power'] > 10) & (df['Power'] < 1000)]

In [13]:
# Оставляем только допустимые значения в RegistrationMonth
df = df[df['RegistrationMonth'].between(1, 12)]

In [14]:
search_losses(df)

Количество дубликатов: 22868

Количество пропусков:
Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Kilometer            0
RegistrationMonth    0
FuelType             0
Brand                0
Repaired             0
dtype: int64



In [15]:
# Удаляем дубликаты
df = df.drop_duplicates()

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260184 entries, 1 to 354368
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Price              260184 non-null  int64 
 1   VehicleType        260184 non-null  object
 2   RegistrationYear   260184 non-null  int64 
 3   Gearbox            260184 non-null  object
 4   Power              260184 non-null  int64 
 5   Model              260184 non-null  object
 6   Kilometer          260184 non-null  int64 
 7   RegistrationMonth  260184 non-null  int64 
 8   FuelType           260184 non-null  object
 9   Brand              260184 non-null  object
 10  Repaired           260184 non-null  object
dtypes: int64(5), object(6)
memory usage: 23.8+ MB


In [17]:
# Удаляем автомобили с ценой 0
df = df[df['Price'] > 0]

***Вывод по предобработке данных***


***1. Обнаружение и обработка пропусков***

- В столбцах `VehicleType`, `Gearbox`, `Model`, `FuelType`, `Repaired` обнаружены пропуски.
- Заполнил их значением "unknown", чтобы сохранить данные и не удалять полезную информацию.

***Результат:***
    Теперь во всех строках нет пропусков, и модели машин с неизвестными характеристиками не удалены.


***2. Удаление неинформативных признаков***

- Удалил `DateCrawled`, `DateCreated`, `LastSeen`, `NumberOfPictures`, `PostalCode`, так как они не влияют на стоимость авто.

***Результат:***
    Датасет стал чище – теперь в нем только нужные признаки.


***3. Фильтрация аномалий***

- Ограничил `RegistrationYear` до `1900–2016`.
- Удалил аномальные значения `Power` (оставил `10–1000 л.с.`, убрал нулевую мощность и нереалистично высокие значения).
- Отфильтровал `RegistrationMonth`, оставив только корректные значения `1–12` (убрал нули).

***Результат:***
    Теперь в данных нет аномально старых или новых машин, нет автомобилей с нулевой мощностью или некорректными месяцами регистрации.


***4. Проверка и удаление дубликатов***

- После удаления неинформативных столбцов обнаружил `22 868 дубликатов`.
- Удалил их, чтобы избежать повторов и улучшить качество модели.

***Результат:***
    Дубликаты полностью удалены.


***5. Проверка и очистка цен***

- Обнаружил 4 170 автомобилей с ценой 0 евро. Удалил их, так как это аномалия.
- Минимальная цена теперь 1 евро.

***Результат:***
    Теперь все машины имеют реалистичную цену, что должно повлиять на качество предсказаний.

In [18]:
target = 'Price'
features = df.drop(columns=[target])
target_values = df[target]

# Выделяем числовые характеристики и категориальные
numeric_features = ['RegistrationYear', 'Power', 'Kilometer', 'RegistrationMonth']
categorical_features = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']

# Разделяем датасет на тестовые и тренировочные подвыборки
X_train, X_test, y_train, y_test = train_test_split(features, target_values, test_size=0.2, random_state=42)

# OHE с исправлением дамми-ловушки
encoder = OneHotEncoder(handle_unknown='ignore', drop='first', sparse=False)
X_train_cat = encoder.fit_transform(X_train[categorical_features])
X_test_cat = encoder.transform(X_test[categorical_features])

# SCALE
scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train[numeric_features])
X_test_num = scaler.transform(X_test[numeric_features])

# Собираем тренировочную и тестовую выборки
X_train_final = np.hstack((X_train_num, X_train_cat))
X_test_final = np.hstack((X_test_num, X_test_cat))

In [19]:
X_train_final.shape, X_test_final.shape

((204811, 311), (51203, 311))

## Обучение моделей

In [20]:
# Функция для обучения модели и оценки времени выполнения
"""def train_and_evaluate(model, X_train, y_train, X_val, y_val):
    start_time = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_time

    start_time = time.time()
    y_pred_train = model.predict(X_train)
    predict_time = time.time() - start_time

    y_pred_val = model.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred_val, squared=False)

    return train_time, predict_time, rmse"""

'def train_and_evaluate(model, X_train, y_train, X_val, y_val):\n    start_time = time.time()\n    model.fit(X_train, y_train)\n    train_time = time.time() - start_time\n\n    start_time = time.time()\n    y_pred_train = model.predict(X_train)\n    predict_time = time.time() - start_time\n\n    y_pred_val = model.predict(X_val)\n    rmse = mean_squared_error(y_val, y_pred_val, squared=False)\n\n    return train_time, predict_time, rmse'

In [21]:
# Выделяем модели и их гиперпараметры
models_params = {
    "Linear Regression": (LinearRegression(), {}),

    "Random Forest": (RandomForestRegressor(random_state=42), {
        "n_estimators": [10, 25, 50],
        "max_depth": [None, 7]
    }),

    "LightGBM": (lgb.LGBMRegressor(random_state=42), {
        "n_estimators": [10, 25, 50],
        "learning_rate": [0.1, 0.05]
    })# Убрал CatBoost, так как очень долго выполняется ячейка
}

In [22]:
# Перебираем модели, обучаем и подбираем лучшие параметры, но с условием на проверку параметров
best_models = {}
for model_name, (model, params) in models_params.items():
    print(f"Обучаем {model_name}...")

    start_time = time.time()  # Засекаем время обучения

    if params:
        search = GridSearchCV(model, params, cv=3, scoring="neg_root_mean_squared_error", n_jobs=-1)
        search.fit(X_train_final, y_train)
        best_model = search.best_estimator_
        best_score = -search.best_score_
    else:
        model.fit(X_train_final, y_train)
        best_model = model
        best_score = mean_squared_error(y_train, model.predict(X_train_final), squared=False)

    train_time = time.time() - start_time  # Время обучения

    # Засекаем время предсказания
    start_pred = time.time()
    _ = best_model.predict(X_train_final)
    predict_time = time.time() - start_pred

    best_models[model_name] = {
        "best_model": best_model,
        "best_score": best_score,
        "train_time": train_time,
        "predict_time": predict_time
    }

Обучаем Linear Regression...
Обучаем Random Forest...
Обучаем LightGBM...


In [23]:
for name, result in best_models.items():
    print(f"{name}: Лучший RMSE = {result['best_score']:.2f}, Время обучения = {result['train_time']:.2f} сек, Время предсказания = {result['predict_time']:.4f} сек")

Linear Regression: Лучший RMSE = 2606.55, Время обучения = 43.80 сек, Время предсказания = 0.3054 сек
Random Forest: Лучший RMSE = 1675.20, Время обучения = 1926.94 сек, Время предсказания = 7.0281 сек
LightGBM: Лучший RMSE = 1749.81, Время обучения = 633.83 сек, Время предсказания = 1.2198 сек


Качество предсказаний (`RMSE`, чем меньше — тем лучше):

`Linear Regression`: 2606.55 (худший результат)

`Random Forest`: 1675.20(лучший результат)

`LightGBM`: 1749.81 

## Анализ моделей

In [24]:
# Сравнение моделей по 3 критериям
model_performance = {}
for name, result in best_models.items():
    best_model = result["best_model"]

    # Засекаем время обучения
    start_train = time.time()
    best_model.fit(X_train_final, y_train)
    train_time = time.time() - start_train

    # Засекаем время предсказания
    start_pred = time.time()
    _ = best_model.predict(X_train_final)
    predict_time = time.time() - start_pred

    # RMSE на кросс-валидации
    rmse_val = result["best_score"]

    # Записываем в таблицу
    model_performance[name] = {
        "Время обучения (сек)": train_time,
        "Время предсказания (сек)": predict_time,
        "RMSE": rmse_val
    }

In [26]:
# Создаём DataFrame с результатами
performance_df = pd.DataFrame(model_performance).T
performance_df

,Время обучения (сек),Время предсказания (сек),RMSE
Linear Regression,42.468408,0.179172,2606.554856
Random Forest,323.579086,6.627192,1675.203871
LightGBM,71.309815,1.299979,1749.810681


In [27]:
# Выбираем лучшую модель по RMSE
best_model_name = performance_df["RMSE"].idxmin()
best_model = best_models[best_model_name]["best_model"]

In [28]:
print(f"Лучшая модель: {best_model_name}")

Лучшая модель: Random Forest


In [29]:
start_test = time.time()
y_test_pred = best_model.predict(X_test_final)
test_time = time.time() - start_test

In [30]:
rmse_test = mean_squared_error(y_test, y_test_pred, squared=False)
print(f"Финальное RMSE на тестовой выборке ({best_model_name}): {rmse_test:.2f}")
print(f"Время предсказания на тесте: {test_time:.4f} секунд")

Финальное RMSE на тестовой выборке (Random Forest): 1643.49
Время предсказания на тесте: 1.6529 секунд


***1. Linear Regression***

- Самая быстрая модель, но низкая точность (`RMSE = 2606.55`).
- Обучается быстро (`42.47 сек`).
- Предсказывает мгновенно (`0.18 сек`).
- ***Не подходит из-за высокого RMSE.***

***2. Random Forest***

- Лучшее качество предсказаний (`RMSE = 1675.20`).
- Финальное `RMSE` на тесте: `1643.49`.
- Долгое обучение (`323.58 сек`).
- Медленное предсказание (`6.63 сек`).
- ***Лучшая модель по точности, но может быть тяжёлой в инференсе.***

***3. LightGBM***

- Быстрее обучается, чем `Random Forest` (`71.31 сек`).
- Достаточно точная (`RMSE = 1749.81`).
- Быстро предсказывает (`1.30 сек`).
- ***Отличная альтернатива Random Forest, если важна скорость работы.***

###### Итоговая рекомендация

- ***Random Forest*** наилучшая точность (`RMSE = 1643.49` на `X_test`).
- ***LightGBM*** быстрее работает, но чуть менее точен (`RMSE = 1749.81`).

Если приоритет — `точность`, используем `Random Forest`.
Если важна `скорость` работы, `LightGBM` может быть лучшим вариантом.